# 8. Asynchronous I/O
## Shared CPU--I/O Workload

- will create another toy problem
    - which we have a CPU-bound problem that needs to communicate frequently with a database to save results
    - CPU workload can be anything
        - e.g.) take bcrypt hash of a random string with larger and larger workload factors to increase the amount of CPU-bound work
    - represents any sort of problem in which a program has heavy calculations to do, and
    - the results must be stored in a database, potentially with a hravy I/O penalty
- restrictions
    - it has an HTTP API
    - response time are on the order of 100 ms
    - database can satisfy many requests at a time
- the database response time is deliberately chosen to be higher than usual

|Difficulty parameter|8|10|11|12|
|---|---|---|---|---|
|Search per iteration|0.0156|0.0623|0.1244|0.2487|



In [5]:
import random 
import string
import bcrypt
import requests
import asyncio
import aiohttp

UTF_8 = "utf8"
URL = "http://127.0.0.1:8080/add"

## Serial

- some simple code that calculates the `bcrypt` hash of a string and makes a request to the database's http API

In [3]:

def do_task(difficulty):
    """
    Hash a number 10 character string using bcrypt
    with a specified difficulty rating
    """
    # we generate a random 10-char byte array
    password = ("".join(random.sample(string.ascii_lowercase, 10))
                .encode(UTF_8))
    # the difficulty parameter sets how hard it is to generate the password
    # by increasing the CPU and memory requirements of the hashing algorithm
    salt = bcrypt.gensalt(difficulty)
    result = bcrypt.hashpw(password, salt)
    return result.decode(UTF_8)


def save_result_serial(result):
    url = URL
    response = requests.post(url, data=result)
    return response.json()


def calculate_task_serial(num_iter, task_difficulty):
    for _ in range(num_iter):
        result = do_task(task_difficulty)
        save_result_serial(result)


## Batched Results

In [6]:
class AsyncBatcher(object):
    def __init__(self, batch_size) -> None:
        self.batch_size = batch_size
        self.batch = []
        self.client_session = None
        self.url = URL

    def __enter__(self):
        return self

    def __exit__(self, *args, **kwargs):
        self.flush()

    def save(self, result):
        self.batch.append(result)
        if len(self.batch) == self.batch_size:
            self.flush()

    def flush(self):
        loop = asyncio.get_event_loop()
        loop.run_until_complete(self.__aflush()) # we can start up an event loop just to run a single async function

    async def __aflush(self):
        async with aiohttp.ClientSession() as session:
            tasks = tuple(self.fetch(result, session) for result in self.batch)
            for task in asyncio.as_completed(tasks):
                await task
        self.batch.clear()

    async def fetch(self, result, session):
        async with session.post(self.url, data=result) as response:
            return await response.json()

def calculate_task_batch(num_iter, task_difficulty):
    with AsyncBatcher(100) as batcher:
        for _ in range(num_iter):
            result = do_task(task_difficulty)
            batcher.save(result)
        batcher.flush()




- runtime for difficulty=8 was brought down to 10.21 seconds
- 6.95 times speed-up
- if the db has infinite throughput, we can take advantage that we get only 100 ms penalty when `AsyncBatcher` is full and does a flush
    - best performance is obtained by just saving all the requests to the db
- in reality - throughput is bounded
    - 100 requests a second
    - must flush every 100 results and take the penalty
    - if all results are saved at the end and issued all at once
        - the server would only procress 100/time
            - extra penalty
- if throughput is so limited
    - might as well run the serial code
- pipelining - this mechanism of batching results
    - can help tremendously 

## Full Async

- full async solution might be preferable
- if CPU task is part of a larger I/O bound program - such as an HTTP server
- if an API service, in response to some of its end points has to perform heavy computations
    - we still want the API to be able to handle concurrent requests and be performant
    - we also want the CPU task to run quickly

In [ ]:
def save_result_aiohttp(client_session):
    sem = asyncio.Semaphore(100)

    async def saver(result):
        nonlocal sem, client_session
        url = URL
        async with sem:
            async with client_session.post(url, data=result) as response:
                return await response.json()

    return saver

async def calculate_task_aiohttp(num_iter, task_difficulty):
    tasks = []
    async with aiohttp.ClientSession() as client_session:
        saver = save_result_aiohttp(client_session)
        for _ in range(num_iter):
            result = do_task(task_difficulty)
            # instead of await db save immediately
            # queue it into the event loop using asyncio.create_task
            task = asyncio.create_task(saver(result))
            tasks.append(task)
            # we pause the main function to allow the event loop to take care of any pending tasks
            # in general, this happens every time when an await statement is run
            # but we generally don't await in CPU-bound code
            # so we need a way to force the function to defer execution to the event loop
            # try to issue it at any loop that we expect to iterate every 50-100 ms 
            await asyncio.sleep(0)
        # wait for any tasks that haven't completed yet
        await asyncio.wait(tasks)